## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import Google_API_KEY

# Configure gmaps API key
gmaps.configure(api_key=Google_API_KEY)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..", "Weather_Database", "WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Loveland,US,2022-07-31 23:52:35,40.3978,-105.0750,303.51,36,0,5.66,clear sky
1,1,Smithers,CA,2022-07-31 23:53:10,54.7804,-127.1743,300.33,44,20,2.57,few clouds
2,2,Shenjiamen,CN,2022-07-31 23:54:15,29.9576,122.2980,301.82,81,100,3.63,overcast clouds
3,3,Busselton,AU,2022-07-31 23:50:39,-33.6500,115.3333,286.99,94,90,12.69,light rain
4,4,Georgetown,MY,2022-07-31 23:48:00,5.4112,100.3354,300.31,87,40,4.12,thunderstorm


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_t = float(input("What is the minimun temperature for your vacation in Kelvin? "))
max_t = float(input("What is the maximun temperature for your vacation in Kelvin? "))


What is the minimun temperature for your vacation in Kelvin? 295
What is the maximun temperature for your vacation in Kelvin? 298


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_temp_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_t) & (city_data_df["Max Temp"] <= max_t)]
cities_temp_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
12,12,Rikitea,PF,2022-07-31 23:50:41,-23.1203,-134.9692,295.57,74,100,10.35,overcast clouds
16,16,Yaan,NG,2022-07-31 23:54:17,7.3833,8.5667,295.00,96,99,3.53,overcast clouds
23,23,Port Hardy,CA,2022-07-31 23:56:05,50.6996,-127.4199,295.25,68,75,5.14,fog
24,24,Bar Harbor,US,2022-07-31 23:56:05,44.3876,-68.2039,295.34,63,51,4.11,broken clouds
26,26,Sao Filipe,CV,2022-07-31 23:56:06,14.8961,-24.4956,297.52,75,38,2.72,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(cities_temp_df))
cities_temp_df.count()

114


City_ID                114
City                   114
Country                114
Date                   114
Lat                    114
Lng                    114
Max Temp               114
Humidity               114
Cloudiness             114
Wind Speed             114
Current Description    114
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_temp_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

114


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
12,12,Rikitea,PF,2022-07-31 23:50:41,-23.1203,-134.9692,295.57,74,100,10.35,overcast clouds
16,16,Yaan,NG,2022-07-31 23:54:17,7.3833,8.5667,295.00,96,99,3.53,overcast clouds
23,23,Port Hardy,CA,2022-07-31 23:56:05,50.6996,-127.4199,295.25,68,75,5.14,fog
24,24,Bar Harbor,US,2022-07-31 23:56:05,44.3876,-68.2039,295.34,63,51,4.11,broken clouds
26,26,Sao Filipe,CV,2022-07-31 23:56:06,14.8961,-24.4956,297.52,75,38,2.72,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Rikitea,PF,295.57,overcast clouds,-23.1203,-134.9692,
16,Yaan,NG,295.00,overcast clouds,7.3833,8.5667,
23,Port Hardy,CA,295.25,fog,50.6996,-127.4199,
24,Bar Harbor,US,295.34,broken clouds,44.3876,-68.2039,
26,Sao Filipe,CV,297.52,scattered clouds,14.8961,-24.4956,
36,Sembakung,ID,296.69,overcast clouds,3.8167,117.2500,
39,Lagoa,PT,295.63,clear sky,39.0500,-27.9833,
44,Bemidji,US,295.58,overcast clouds,47.4736,-94.8803,
46,Jamestown,US,296.77,overcast clouds,42.0970,-79.2353,
51,Puerto Ayora,EC,295.12,broken clouds,-0.7393,-90.3518,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": Google_API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found!! Skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.
Hotel not found!! Skipping the city.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
12,Rikitea,PF,295.57,overcast clouds,-23.1203,-134.9692,People ThankYou
23,Port Hardy,CA,295.25,fog,50.6996,-127.4199,Airport Inn-Port Hardy
24,Bar Harbor,US,295.34,broken clouds,44.3876,-68.2039,Atlantic Oceanside Hotel & Event Center
26,Sao Filipe,CV,297.52,scattered clouds,14.8961,-24.4956,Hotel Xaguate
39,Lagoa,PT,295.63,clear sky,39.0500,-27.9833,Casa Das Faias
44,Bemidji,US,295.58,overcast clouds,47.4736,-94.8803,Hampton Inn & Suites Bemidji
46,Jamestown,US,296.77,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
51,Puerto Ayora,EC,295.12,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
56,Laurel,US,296.48,moderate rain,39.0993,-76.8483,Days Inn & Suites by Wyndham Laurel Near Fort ...
62,Norman Wells,CA,296.17,few clouds,65.2820,-126.8329,Heritage Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))